In [31]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim import corpora
from gensim import models
import MeCab
from gensim.models import word2vec

In [32]:
# Read recipe inputs
ramen_dictionary_path = dataiku.Folder("POe5uF4H").get_path() + "/ramen_dictionary"
your_trip_advisor = dataiku.Dataset("your_trip_advisor")
df = your_trip_advisor.get_dataframe()

In [33]:
folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [25]:
w2v_folder = dataiku.Folder("m9JZdV7b").get_path()
text_folder = dataiku.Folder("aLTWBozg").get_path()
wakati_folder = dataiku.Folder("0kM5kXKs").get_path()
tagger_path = '-Owakati -d ' + wakati_folder

In [26]:
tagger = MeCab.Tagger(tagger_path)#タグはMeCab.Tagger（neologd辞書）を使用
tagger.parse('')

'\n'

In [27]:
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [28]:
wakati_ramen_text = []
for i in df['jp']:
    txt = tokenize(i, 2, 10000, True)
    wakati_ramen_text.append(txt)

In [41]:
#[w for w in sublist in wakati_ramen_text]
vocab = [w for sublist in wakati_ramen_text for w in sublist]

In [46]:
vectors = [ramen_model.wv[word] for word in vocab]

KeyError: "word 'フランス' not in vocabulary"

In [9]:
dictionary = corpora.Dictionary.load(ramen_dictionary_path)

In [10]:
dictionary.num_docs, dictionary.num_pos

(211, 251395)

In [18]:
reviews_concat = '。'.join(df['jp'].values)

In [0]:
# reviews_cleaned = ...

In [19]:
corpus = list(map(dictionary.doc2bow, reviews_concat))

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

trip_advisor_tf_idf_df = your_trip_advisor_df # For this sample code, simply copy input to output


# Write recipe outputs
trip_advisor_tf_idf = dataiku.Dataset("trip_advisor_tf_idf")
trip_advisor_tf_idf.write_with_schema(trip_advisor_tf_idf_df)